In [1]:
# imports
from copy import copy, deepcopy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import linear_model
import pandas as pd 
import numpy as np
from sklearn import metrics  
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelEncoder
import random
import string
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelBinarizer


%matplotlib inline

In [2]:
# Load the Wine Dataset
df_wine = pd.read_csv("../Country_vertical/df_wine_vertical.csv", encoding = 'utf8', index_col=0)

In [3]:
df_wine.head(1)

,country,province,region,price,variety,points,year_of_wine
0,US,Oregon,Willamette Valley,14.0,Pinot Gris,87,2013


# Data Preparition

In [4]:
# DEFINE X and Y VARIABLES
df_X = df_wine.drop('points', axis=1)
df_X = df_X.drop('region', axis=1)
df_X = df_X.drop('price', axis=1)
df_X = df_X.drop('variety', axis=1)
df_X = df_X.drop('year_of_wine', axis=1)
df_X = df_X.drop('country', axis=1)
df_Y = df_wine[['points']]

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.3, random_state=0)

In [5]:
# GROUND TRUTH
gt_test_training = pd.concat([X_test, X_train])
gt_training = X_train.copy()
gt_test = X_test.copy()


# INTRODUCE MISSING VALUES

In [6]:
def add_missing_values(df, column_name, ratio):
    ix = [(row, col) for row in range(df[column_name].shape[0]) for col in range(df.shape[1])]
    for row, col in random.sample(ix, int(round(ratio*len(ix)))):
        if is_string_dtype(df[column_name]):
            df.iat[row, col] = 'X'
        elif is_numeric_dtype(agg[y]):
            df.iat[row, col] = 0
    
def dtr(X_train, y_train, X_test):
    # Decision Tree Regressor
    regressor = DecisionTreeRegressor()  
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    
    # The evaluation metrics
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    R2 = r2_score(y_test, y_pred)*100

    results = {"MAE" : round(MAE, 3),
              "MSE" : round(MSE, 3),
              "RMSE" : round(RMSE, 3),
              "R2": round(R2, 3)}
    return results

In [8]:
# DECISION TREE WITHOUT MISSING VALUES
print('BASE RESULTS:')
lb = LabelBinarizer()
X_train = pd.DataFrame(lb.fit_transform(X_train['province']), columns=lb.classes_)
X_test = pd.DataFrame(lb.transform(X_test['province']), columns=lb.classes_)
print(dtr(X_train, y_train, X_test))
X_train = gt_training
X_test = gt_test

BASE RESULTS:
{'MAE': 2.396, 'MSE': 8.683, 'RMSE': 2.947, 'R2': 7.769}


In [10]:
# MISSING VALUES IN TEST
ratios = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
results_region_missing_in_test = {}
for ratio in ratios:
    add_missing_values(X_test, 'province', ratio)
    lb = LabelBinarizer()
    X_train = pd.DataFrame(lb.fit_transform(X_train['province']), columns=lb.classes_)
    X_test = pd.DataFrame(lb.transform(X_test['province']), columns=lb.classes_)
    results_region_missing_in_test[ratio] = dtr(X_train, y_train, X_test)
    X_train = gt_training
    X_test = gt_test

In [11]:
results_region_missing_in_test

{0.1: {'MAE': 2.506, 'MSE': 9.452, 'RMSE': 3.074, 'R2': -0.401},
 0.2: {'MAE': 2.505, 'MSE': 9.467, 'RMSE': 3.077, 'R2': -0.56},
 0.3: {'MAE': 2.506, 'MSE': 9.452, 'RMSE': 3.074, 'R2': -0.406},
 0.4: {'MAE': 2.506, 'MSE': 9.452, 'RMSE': 3.074, 'R2': -0.405},
 0.5: {'MAE': 2.506, 'MSE': 9.452, 'RMSE': 3.074, 'R2': -0.405},
 0.6: {'MAE': 2.506, 'MSE': 9.452, 'RMSE': 3.074, 'R2': -0.406},
 0.7: {'MAE': 2.504, 'MSE': 9.467, 'RMSE': 3.077, 'R2': -0.56},
 0.8: {'MAE': 2.506, 'MSE': 9.452, 'RMSE': 3.074, 'R2': -0.406},
 0.9: {'MAE': 2.506, 'MSE': 9.452, 'RMSE': 3.074, 'R2': -0.406}}

In [16]:
# MISSING VALUES IN TRAINING
ratios = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
results_region_missing_in_train = {}
for ratio in ratios:
    add_missing_values(X_train, 'province', ratio)
    lb = LabelBinarizer()
    X_train = pd.DataFrame(lb.fit_transform(X_train['province']), columns=lb.classes_)
    X_test = pd.DataFrame(lb.transform(X_test['province']), columns=lb.classes_)
    results_region_missing_in_train[ratio] = dtr(X_train, y_train, X_test)
    X_train = gt_training
    X_test = gt_test

In [17]:
results_region_missing_in_train

{0.1: {'MAE': 2.514, 'MSE': 9.414, 'RMSE': 3.068, 'R2': -0.0},
 0.2: {'MAE': 2.514, 'MSE': 9.414, 'RMSE': 3.068, 'R2': 0.0},
 0.3: {'MAE': 2.516, 'MSE': 9.414, 'RMSE': 3.068, 'R2': 0.0},
 0.4: {'MAE': 2.516, 'MSE': 9.413, 'RMSE': 3.068, 'R2': 0.0},
 0.5: {'MAE': 2.516, 'MSE': 9.413, 'RMSE': 3.068, 'R2': 0.0},
 0.6: {'MAE': 2.516, 'MSE': 9.413, 'RMSE': 3.068, 'R2': 0.0},
 0.7: {'MAE': 2.516, 'MSE': 9.414, 'RMSE': 3.068, 'R2': 0.0},
 0.8: {'MAE': 2.516, 'MSE': 9.414, 'RMSE': 3.068, 'R2': -0.0},
 0.9: {'MAE': 2.516, 'MSE': 9.415, 'RMSE': 3.068, 'R2': -0.0}}

In [36]:
len(df_wine.region.unique().tolist())

1005

# Precision & Recall

In [132]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(gt_test_training['country'].values, 
                                dirty_test_training['country'].values, 
                                average='weighted')

NameError: name 'dirty_test_training' is not defined

# HPO with ground truth

In [133]:
# # Using Randomized Search for hyperparameter optimisation

# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# from scipy.stats import randint as sp_randint
# regressor3 = DecisionTreeRegressor()
# param_dist = {'max_depth': sp_randint(2,16),
#               'min_samples_split': sp_randint(2,16)}

# n_iter_search = 20
# clfrs = RandomizedSearchCV(regressor3, 
#                                    param_distributions=param_dist,
#                                    scoring='neg_mean_squared_error',
#                                    cv=5 , n_jobs=1, verbose=1,
#                                    n_iter = n_iter_search)
# clfrs.fit(df_X, df_Y)
# clfrs.best_params_, clfrs.best_score_